# Master Thesis 2nd Version of a STT Model with new State of the Art: Whisper

**Author**: Karin Thommen

**Date**: April 2023 


---

**Content of the Notebook**:  Fine-tuning and Training of OpenAi Whisper ASR Model

---

## Step 1: Import and Setup

In [6]:
%%capture
!pip install datasets>=2.6.1
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio
!pip install audio-metadata

In [7]:
pip install "dill<0.3.5"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.14 requires dill>=0.3.6, but you have dill 0.3.4 which is incompatible.


In [8]:
import pandas as pd
import os
import transformers

from datasets.fingerprint import Hasher
import pickle
import dill

from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json

import IPython.display as ipd
import numpy as np
import random

import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

import audio_metadata

from datasets import load_dataset, Audio, load_metric, load_from_disk, DatasetDict
from datasets import Dataset, Sequence

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

from transformers import WhisperTokenizer
from transformers import WhisperTokenizerFast
from transformers import WhisperProcessor
from transformers import WhisperFeatureExtractor


In [9]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

## Step 2: Load Data

In [10]:
# Build connection to data folder on GDrive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
# load train, test and validation files 
train=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/train.tsv",sep='\t')
test=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/test.tsv",sep='\t')
valid=pd.read_csv("/content/drive/MyDrive/Masterarbeit/Data/SDS-200/SDS-200 Corpus/splits/valid.tsv",sep='\t')

In [35]:
!git clone 

^C


In [12]:
# show training data 
train.head()

,clip_id,clip_path,sentence,clip_created_at,clip_is_valid,sentence_id,sentence_source,client_id,zipcode,canton,user_mean_clip_quality,clip_n_votes_correct,clip_n_votes_false,clip_n_times_reported,sentence_n_times_reported,age,gender,duration,user_sentences,continuous_client_index
0,127972,8587421a-f201-4e8a-95a9-e3d3f2473f5e/0f97ad53f...,Dies wird inzwischen durch Forschungsergebniss...,2021-08-25 08:20:01,NaN,0f97ad53f8c704aa2d9b0200bcf5718bacae8b58128106...,cv_sentences,8587421a-f201-4e8a-95a9-e3d3f2473f5e,5000.0,AG,0.9011,0,0,0,0,fourties,male,4.176,1038,NaN
1,32935,b5685887-4f42-40dd-8e46-349bcb5ff4aa/034cb4228...,Des Weiteren gibt es noch eine Abteilung für E...,2021-07-11 13:51:12,True,034cb4228965e95835de3cf62c129b148e5425f704a1ba...,cv_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,2,0,0,0,thirties,male,3.960,11696,16.0
2,124203,b5685887-4f42-40dd-8e46-349bcb5ff4aa/2c5681ae9...,Und wo ich jetzt bin.,2021-08-24 14:24:04,NaN,2c5681ae96fe29fb5db2e1bfb523bb9e413a05d2dbec87...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,2.124,11696,NaN
3,85001,956d47bc-29f0-4ed7-ad31-f39df394f945/1e3d7a806...,Gegen Stuttgart 21 wird seit Monaten heftig pr...,2021-08-03 08:55:54,NaN,1e3d7a8060fc453c110d28aa0e2abe2031cf34d0ad5e3e...,tamedia_sentences,956d47bc-29f0-4ed7-ad31-f39df394f945,8810.0,ZH,0.9764,0,0,0,0,fourties,male,5.040,13365,NaN
4,113554,b5685887-4f42-40dd-8e46-349bcb5ff4aa/27f4be70b...,Zu mehr Freiheit wird die neue Religion aber n...,2021-08-20 19:50:09,NaN,27f4be70b7e7134e9a65ff9ec14b27577a9f4b340d8022...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,4.212,11696,NaN


In [13]:
train = train[train['duration'] <= 5.0]
train = train.reset_index()
train = train.drop(columns=["index"])
#train = train.sample(n=7000)
#train = train.reset_index()
#train = train.drop(columns=["index"])
len(train)

84771

In [14]:
train.head()

,clip_id,clip_path,sentence,clip_created_at,clip_is_valid,sentence_id,sentence_source,client_id,zipcode,canton,user_mean_clip_quality,clip_n_votes_correct,clip_n_votes_false,clip_n_times_reported,sentence_n_times_reported,age,gender,duration,user_sentences,continuous_client_index
0,127972,8587421a-f201-4e8a-95a9-e3d3f2473f5e/0f97ad53f...,Dies wird inzwischen durch Forschungsergebniss...,2021-08-25 08:20:01,NaN,0f97ad53f8c704aa2d9b0200bcf5718bacae8b58128106...,cv_sentences,8587421a-f201-4e8a-95a9-e3d3f2473f5e,5000.0,AG,0.9011,0,0,0,0,fourties,male,4.176,1038,NaN
1,32935,b5685887-4f42-40dd-8e46-349bcb5ff4aa/034cb4228...,Des Weiteren gibt es noch eine Abteilung für E...,2021-07-11 13:51:12,True,034cb4228965e95835de3cf62c129b148e5425f704a1ba...,cv_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,2,0,0,0,thirties,male,3.960,11696,16.0
2,124203,b5685887-4f42-40dd-8e46-349bcb5ff4aa/2c5681ae9...,Und wo ich jetzt bin.,2021-08-24 14:24:04,NaN,2c5681ae96fe29fb5db2e1bfb523bb9e413a05d2dbec87...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,2.124,11696,NaN
3,113554,b5685887-4f42-40dd-8e46-349bcb5ff4aa/27f4be70b...,Zu mehr Freiheit wird die neue Religion aber n...,2021-08-20 19:50:09,NaN,27f4be70b7e7134e9a65ff9ec14b27577a9f4b340d8022...,tamedia_sentences,b5685887-4f42-40dd-8e46-349bcb5ff4aa,8404.0,ZH,0.9786,0,0,0,0,thirties,male,4.212,11696,NaN
4,124119,956d47bc-29f0-4ed7-ad31-f39df394f945/0f1e14cda...,Sie hat bisher alle Weltmeisterschaften gewonnen.,2021-08-24 14:13:08,NaN,0f1e14cdaad422d35523e043a55721764d5dcd8f3767df...,cv_sentences,956d47bc-29f0-4ed7-ad31-f39df394f945,8810.0,ZH,0.9764,0,0,0,0,fourties,male,3.384,13365,NaN


In [18]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [19]:
# load tokenizer form Whisper Tokenizer
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-small", task="transcribe")

In [20]:
# load feature extractor 
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-small")

In [ ]:
# checking the meta data of the audio to get the sampling rate of the saved audios 
path_all = "https://huggingface.co/datasets/karinthommen/sds200/main/corpus/"
metadata = audio_metadata.load(path_all + train["clip_path"][0])
print(metadata)

## Step 3: Prepare Dataset and convert it into the correct Format

In [ ]:
def 

In [13]:
def make_audio_dataset(data):
  audio_list = data.clip_path.values.tolist() # convert the column to a list 
  audio_list = [path_all + s for s in audio_list] # add root of path to all pathes in list 

  audio_dataset = Dataset.from_dict({"path": data.clip_path.values.tolist(), "audio": audio_list, "sentence": data.sentence.values.tolist()}).cast_column("audio", Audio(sampling_rate=16_000)) # convert it to a Dataset for later usage 
  
  
  return audio_dataset 


train_audio = make_audio_dataset(train)
valid_audio = make_audio_dataset(valid)
test_audio = make_audio_dataset(test)

# check meta data via newly created Dataset object
train_audio[0]

{'path': '8587421a-f201-4e8a-95a9-e3d3f2473f5e/0f97ad53f8c704aa2d9b0200bcf5718bacae8b58128106d0072432fde948a747.mp3',
 'audio': {'path': '/content/drive/MyDrive/Masterarbeit/Data/SDS-200/export_20211220_clips-001/8587421a-f201-4e8a-95a9-e3d3f2473f5e/0f97ad53f8c704aa2d9b0200bcf5718bacae8b58128106d0072432fde948a747.mp3',
  'array': array([ 0.00000000e+00, -1.45519152e-11,  1.09139364e-11, ...,
          6.61111699e-06,  5.62083915e-06, -7.73885404e-06]),
  'sampling_rate': 16000},
 'sentence': 'Dies wird inzwischen durch Forschungsergebnisse widerlegt.'}

In [40]:
data = DatasetDict()
data["train"] = train_audio
data["test"] = test_audio
data["valid"] = valid_audio
print(data)

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'sentence'],
        num_rows: 84771
    })
    test: Dataset({
        features: ['path', 'audio', 'sentence'],
        num_rows: 3636
    })
    valid: Dataset({
        features: ['path', 'audio', 'sentence'],
        num_rows: 3638
    })
})


In [44]:
# Check if audio loading worked with a random audio and sentence 
rand_int = random.randint(0, len(train)-1)
print(data["train"]["sentence"][rand_int])
ipd.Audio(data=data["train"][rand_int]["audio"]["array"], autoplay=True, rate=16000)

Dabei wurden zwei Beamte verletzt.


In [45]:
# Check sentence, input array shape and sampling rate 
rand_int = random.randint(0, len(data["train"])-1)

print("Target text:", data["train"][rand_int]["sentence"])
print("Input array shape:", data["train"][rand_int]["audio"]["array"].shape)
print("Sampling rate:", data["train"][rand_int]["audio"]["sampling_rate"])

Target text: Grössere Erfolge erzielte er ausschliesslich im Doppel.
Input array shape: torch.Size([78336])
Sampling rate: tensor(16000)


In [46]:
# show sentence decoded with the special characters ( in the format that is needed by whisper )
input_str = data["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")

Input:                 Dies wird inzwischen durch Forschungsergebnisse widerlegt.
Decoded w/ special:    <|startoftranscript|><|transcribe|><|notimestamps|>Dies wird inzwischen durch Forschungsergebnisse widerlegt.<|endoftext|>
Decoded w/out special: Dies wird inzwischen durch Forschungsergebnisse widerlegt.
Are equal:             True


In [17]:
# load processor from Whisper Processor 
processor = WhisperProcessor.from_pretrained("openai/whisper-small", task="transcribe",)

In [47]:
#show format of train dataset 
data["train"][0]

{'path': '8587421a-f201-4e8a-95a9-e3d3f2473f5e/0f97ad53f8c704aa2d9b0200bcf5718bacae8b58128106d0072432fde948a747.mp3',
 'audio': {'path': '/content/drive/MyDrive/Masterarbeit/Data/SDS-200/export_20211220_clips-001/8587421a-f201-4e8a-95a9-e3d3f2473f5e/0f97ad53f8c704aa2d9b0200bcf5718bacae8b58128106d0072432fde948a747.mp3',
  'array': tensor([ 0.0000e+00, -1.4552e-11,  1.0914e-11,  ...,  6.6111e-06,
           5.6208e-06, -7.7389e-06]),
  'sampling_rate': tensor(16000)},
 'sentence': 'Dies wird inzwischen durch Forschungsergebnisse widerlegt.'}

In [25]:
import multiprocessing
cores = multiprocessing.cpu_count() # Count the number of cores 
cores

4

### Step 3a: Prepare Dataset (skip & jump to step 3b if you want to load the dataset directly from disk) 

In [31]:
def prepare_dataset(batch):
    # load and resample audio data from 48 to 16kHz
    audio = batch["audio"]

    # compute log-Mel input features from input audio array 
    batch["input_features"] = feature_extractor(audio["array"], sampling_rate=audio["sampling_rate"]).input_features[0]
    # encode target text to label ids 
    batch["labels"] = tokenizer(batch["sentence"]).input_ids
    return batch

In [48]:
data["train"].info

DatasetInfo(description='', citation='', homepage='', license='', features={'path': Value(dtype='string', id=None), 'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None), 'sentence': Value(dtype='string', id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, config_name=None, version=None, splits=None, download_checksums=None, download_size=None, post_processing_size=None, dataset_size=None, size_in_bytes=None)

In [ ]:
train_prep = data.map(prepare_dataset, remove_columns=data.column_names["train"])

Map:   0%|          | 0/84771 [00:00<?, ? examples/s]

In [ ]:
valid_prep = valid_audio.map(prepare_dataset)

Map:   0%|          | 0/3638 [00:00<?, ? examples/s]

In [ ]:
test_prep = test_audio.map(prepare_dataset)

Map:   0%|          | 0/3636 [00:00<?, ? examples/s]

In [ ]:
test_prep.save_to_disk("/content/drive/MyDrive/Masterarbeit/Data/test.hf")

In [ ]:
valid_prep.save_to_disk("/content/drive/MyDrive/Masterarbeit/Data/valid.hf")

Saving the dataset (0/9 shards):   0%|          | 0/3638 [00:00<?, ? examples/s]

In [ ]:
train_prep.save_to_disk("/content/drive/MyDrive/Masterarbeit/Data/train.hf")

Saving the dataset (0/16 shards):   0%|          | 0/7000 [00:00<?, ? examples/s]

### Step 3b: Load Dataset (skip if new dataset was loaded in Step 3a) 

In [9]:
train_prep = load_from_disk("/content/drive/MyDrive/Masterarbeit/Data/train.hf")

In [10]:
test_prep = load_from_disk("/content/drive/MyDrive/Masterarbeit/Data/test.hf")

In [11]:
valid_prep = load_from_disk("/content/drive/MyDrive/Masterarbeit/Data/valid.hf")

## Fine-Tune & Train Model 

In [14]:
@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [25]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [26]:
import evaluate

metric = evaluate.load("wer")

In [27]:
def compute_metrics(pred):
    pred_ids = pred.predictions
    label_ids = pred.label_ids

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = tokenizer.pad_token_id

    # we do not want to group tokens when computing the metrics
    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    wer = 100 * metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [28]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small").to("cuda")

In [29]:
model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

In [30]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="/content/drive/MyDrive/Masterarbeit/STT_V2_Whisper_1/whisper-small-hi",  # change to a repo name of your choice
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-5,
    warmup_steps=100,
    max_steps=2000,
    gradient_checkpointing=True,
    fp16=True,
    evaluation_strategy="steps",
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=500,
    eval_steps=500,
    logging_steps=25,
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model="wer",
    greater_is_better=False)

In [31]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_prep,
    eval_dataset=valid_prep,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

In [32]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
#torch.cuda.empty_cache()

### Training Process

In [33]:
trainer.train()

    This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
`use_cache = True` is incompatible with gradient checkpointing. Setting `use_cache = False`...


Step,Training Loss,Validation Loss,Wer
500,0.317300,0.669520,206.339936
1000,0.153200,0.666954,543.186856
1500,0.081400,0.689956,392.059248


Step,Training Loss,Validation Loss,Wer
500,0.317300,0.669520,206.339936
1000,0.153200,0.666954,543.186856
1500,0.081400,0.689956,392.059248
2000,0.052300,0.706085,744.425289


TrainOutput(global_step=2000, training_loss=0.3275391261577606, metrics={'train_runtime': 21890.2902, 'train_samples_per_second': 1.462, 'train_steps_per_second': 0.091, 'total_flos': 9.22549807742976e+18, 'train_loss': 0.3275391261577606, 'epoch': 4.57})